# Cleaning and Formatting Olympic Dataset for Upload: 


In [473]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_columns', None)

In [275]:
df = pd.read_csv("/Users/isaiahhollars/Desktop/olympicProject/athlete_events.csv")

## Add Event Gender Column

In [276]:
conditions = [
    (df['Event'].str.contains("Men's")),
    (df['Event'].str.contains("Women's")),
    (df['Event'].str.contains("Mixed")) 
    ]

# create a list of the values we want to assign for each condition
values = ["Men's", "Women's", "Mixed"]

# create a new column and use np.select to assign values to it using our lists as arguments
df['EventGender'] = np.select(conditions, values)

# display updated DataFrame
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,Men's
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,Men's
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Men's
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Men's
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Women's


## Change the Event Column to exclude the gender

In [277]:
conditions = [
    (df['Event'].str.contains("Men's")),
    (df['Event'].str.contains("Women's")),
    (df['Event'].str.contains("Mixed")) 
    ]

# create a list of the values we want to assign for each condition
values = [df['Event'].str.replace("Men's ",''),
          df['Event'].str.replace("Women's ",''),
         df['Event'].str.replace("Mixed " ,',') ]

# create a new column and use np.select to assign values to it using our lists as arguments
df['Event'] = np.select(conditions, values)

# display updated DataFrame
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,NaN,Men's
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Extra-Lightweight,NaN,Men's
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Football,NaN,Men's
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Tug-Of-War,Gold,Men's
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating 500 metres,NaN,Women's


## Take out redundency in Event Column

In [278]:
def removeDouble(row):
    return row['Event'].replace(row['Sport'], '')

def addBack(row):
    if row['Event2']==' ' or row['Event2']=='' or row['Event2']==',':
        return row['Sport'].strip()     
    else: return row['Event2'].replace(',','' ).strip()

    
def fixEmpty(row):
    if row['Event2']=='':
        return row['Sport']
    else: 
        return row['Event2']
    
    
def fixLugeGender(row):
    if row['Event2']=="(Men)'s Doubles":
        return "Men's"
    else:
        return row['EventGender']
    
    
def fixLugeEvent(row):
    if row['Event2']=="(Men)'s Doubles":
        return "Doubles"
    else:
        return row['Event2']


    
df["Event2"] = df.apply(removeDouble, axis=1)  
df['Event2'] = df.apply(addBack, axis=1)
df['Event2'] = df.apply(fixEmpty, axis=1)
df['EventGender']=df.apply(fixLugeGender, axis=1)
df['Event2']=df.apply(fixLugeEvent, axis=1)

df.head()



,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender,Event2
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,NaN,Men's,Basketball
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Extra-Lightweight,NaN,Men's,Extra-Lightweight
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Football,NaN,Men's,Football
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Tug-Of-War,Gold,Men's,Tug-Of-War
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating 500 metres,NaN,Women's,500 metres


## change age, height, weight, to ints for SQL to handel better

In [279]:
df['Age'] = df['Age'].fillna(0)
df['Height'] = df['Height'].fillna(0)
df['Weight'] = df['Weight'].fillna(0)
df['Medal'] = df['Medal'].fillna("None") #also replace null with "None" for medal string 

In [280]:
df['Age'] = df['Age'].astype(int)
df['Height'] = df['Height'].astype(int)
df['Weight'] = df['Weight'].astype(int)

## Now it's time for first and Last names... 

In [281]:
def fname(row): 
    return row['Name'].split(' ')[0]
def lname(row):
    return row['Name'].split(' ')[-1]

In [282]:
df['Ath_FN']=df.apply (lambda row: fname(row), axis=1)
df['Ath_LN']= df.apply (lambda row: lname(row), axis=1)
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender,Event2,Ath_FN,Ath_LN
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,A,Dijiang
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Extra-Lightweight,None,Men's,Extra-Lightweight,A,Lamusi
2,3,Gunnar Nielsen Aaby,M,24,0,0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Football,None,Men's,Football,Gunnar,Aaby
3,4,Edgar Lindenau Aabye,M,34,0,0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Tug-Of-War,Gold,Men's,Tug-Of-War,Edgar,Aabye
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating 500 metres,None,Women's,500 metres,Christine,Aaftink


## make games csv file

In [283]:
dfGames = df[['City','Season','Year']]

In [284]:
dfGames=dfGames.drop_duplicates()
dfGames.describe()


,Year
count,52.000000
mean,1963.115385
std,34.034385
min,1896.000000
25%,1935.000000
50%,1966.000000
75%,1992.000000
max,2016.000000


In [285]:
dfGames['GamesID'] = [i for i in range(1,len(dfGames.index)+1)]

In [287]:
dfGames.head()

,City,Season,Year,GamesID
0,Barcelona,Summer,1992,1
1,London,Summer,2012,2
2,Antwerpen,Summer,1920,3
3,Paris,Summer,1900,4
4,Calgary,Winter,1988,5


In [288]:
dfGames = dfGames[['GamesID','City','Season','Year']] #reorder 

#rename: 
dfGames = dfGames.rename(columns={ 'City':'Games_City','Season':'Games_Season','Year':'Games_Year'})

In [289]:
dfGames.head()

,GamesID,Games_City,Games_Season,Games_Year
0,1,Barcelona,Summer,1992
1,2,London,Summer,2012
2,3,Antwerpen,Summer,1920
3,4,Paris,Summer,1900
4,5,Calgary,Winter,1988


In [290]:
dfGames.to_csv('/Users/isaiahhollars/Desktop/olympicProject/GamesTable', sep='\t', encoding='utf-8', header=True, index=False)

## make athlete csv file

In [350]:
dfAth = df[['ID','Ath_FN','Ath_LN','Sex','Age','Height','Weight','Year']]
dfAth.head(1)

,ID,Ath_FN,Ath_LN,Sex,Age,Height,Weight,Year
0,1,A,Dijiang,M,24,180,80,1992


In [351]:
dfAth.replace(')', value='')
dfAth.replace('(', value='')
dfAth.replace(' ', value='')

,ID,Ath_FN,Ath_LN,Sex,Age,Height,Weight,Year
0,1,A,Dijiang,M,24,180,80,1992
1,2,A,Lamusi,M,23,170,60,2012
2,3,Gunnar,Aaby,M,24,0,0,1920
3,4,Edgar,Aabye,M,34,0,0,1900
4,5,Christine,Aaftink,F,21,185,82,1988
...,...,...,...,...,...,...,...,...
271111,135569,Andrzej,ya,M,29,179,89,1976
271112,135570,Piotr,ya,M,27,176,59,2014
271113,135570,Piotr,ya,M,27,176,59,2014
271114,135571,Tomasz,ya,M,30,185,96,1998


In [352]:
#drop repeats df.drop_duplicates(subset=['brand'])
dfAth.drop_duplicates(subset=['ID'], inplace=True)
dfAth.head(1)

/Users/isaiahhollars/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Ath_FN,Ath_LN,Sex,Age,Height,Weight,Year
0,1,A,Dijiang,M,24,180,80,1992


In [353]:
def yob(row):
    return row['Year']-row['Age']

dfAth['Ath_YOB']= dfAth.apply (lambda row: yob(row), axis=1)

dfAth.head(1)

/Users/isaiahhollars/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID,Ath_FN,Ath_LN,Sex,Age,Height,Weight,Year,Ath_YOB
0,1,A,Dijiang,M,24,180,80,1992,1968


In [354]:
dfAth = dfAth.astype({"ID": int, "Ath_YOB": int, 'Height':int, 'Weight':int})

In [355]:
dfAth.dtypes

ID          int64
Ath_FN     object
Ath_LN     object
Sex        object
Age         int64
Height      int64
Weight      int64
Year        int64
Ath_YOB     int64
dtype: object

In [356]:
#rename: 
dfAth = dfAth.rename(columns={ 'ID':'AthID','Sex':'Ath_Sex','Age':'Ath_Age','Height':'Ath_Height','Weight':'Ath_Weight'})
dfAth.head(1)

,AthID,Ath_FN,Ath_LN,Ath_Sex,Ath_Age,Ath_Height,Ath_Weight,Year,Ath_YOB
0,1,A,Dijiang,M,24,180,80,1992,1968


In [357]:
#reorder
dfAth = dfAth[['AthID','Ath_FN','Ath_LN','Ath_Sex','Ath_YOB','Ath_Height','Ath_Weight']] #reorder 
dfAth.head(1)

,AthID,Ath_FN,Ath_LN,Ath_Sex,Ath_YOB,Ath_Height,Ath_Weight
0,1,A,Dijiang,M,1968,180,80


In [358]:
dfAth['Ath_FN'] = df['Ath_FN'].str.strip()
dfAth['Ath_LN'] = df['Ath_LN'].str.strip()

In [359]:
dfAth.to_csv('/Users/isaiahhollars/Desktop/olympicProject/AthTable', sep='\t', encoding='utf-8', header=True, index=False)

In [360]:
dfAth.tail(10)

,AthID,Ath_FN,Ath_LN,Ath_Sex,Ath_YOB,Ath_Height,Ath_Weight
271101,135562,Milan,Zyka,M,1948,173,68
271102,135563,Olesya,Zykina,F,1981,171,64
271104,135564,Yevgeny,Zykov,M,1980,172,65
271105,135565,Fernando,Zylberberg,M,1977,168,76
271107,135566,James,Zylker,M,1951,175,75
271108,135567,Aleksandr,Zyuzin,M,1976,183,72
271110,135568,Olga,Zyuzkova,F,1983,171,69
271111,135569,Andrzej,ya,M,1947,179,89
271112,135570,Piotr,ya,M,1987,176,59
271114,135571,Tomasz,ya,M,1968,185,96


## TEAM table

In [522]:
regions = pd.read_csv("/Users/isaiahhollars/Desktop/olympicProject/noc_regions.csv")
regions.head(1)

,NOC,region,notes
0,AFG,Afghanistan,NaN


In [523]:
regions=regions[['NOC','region']]

In [524]:
#rename: 
regions = regions.rename(columns={ 'NOC':'TeamNOC','region':'Team_Region'})

In [525]:
regions.drop_duplicates(subset=['TeamNOC'],inplace=True)
regions.head(1)

,TeamNOC,Team_Region
0,AFG,Afghanistan


In [526]:
regions.shape

(230, 2)

In [528]:
temp = df.rename(columns={'NOC':'TeamNOC'})
temp=temp[['TeamNOC']]
temp.drop_duplicates(subset=['TeamNOC'], inplace = True)
temp.head(5)

,TeamNOC
0,CHN
2,DEN
4,NED
10,USA
28,FIN


In [529]:
temp.shape

(230, 1)

In [534]:
#bigdata = pd.concat([data1, data2], ignore_index=True, sort=False)
join = pd.concat([regions,temp],ignore_index=True, sort=False)
join.shape
join.head()

,TeamNOC,Team_Region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [539]:
join.drop_duplicates(subset=['TeamNOC'],inplace=True)
join[join['Team_Region'].isna()]

,TeamNOC,Team_Region
168,ROT,NaN
208,TUV,NaN
213,UNK,NaN
295,SGP,NaN


In [541]:
#now we have the extra entry we were missing in the NOC
join.shape

(231, 2)

In [495]:
#df[df['column name'].isna()]
#temp2[temp2['Team_Region'].isna()]

,TeamNOC,Team_Region
264737,ROT,NaN
270680,UNK,NaN
270682,TUV,NaN


In [497]:
#temp2.shape

(229, 2)

In [542]:
join.to_csv('/Users/isaiahhollars/Desktop/olympicProject/TeamTable', sep='\t', encoding='utf-8', header=True, index=False)

## Event Table - references the Games table

In [439]:
df.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender,Event2,Ath_FN,Ath_LN
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,A,Dijiang
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Extra-Lightweight,None,Men's,Extra-Lightweight,A,Lamusi


In [440]:
ev = df[['Sport','Event2','EventGender','Year','Season']]
ev = ev.rename(columns={ 'Sport':'Ev_Sport','Event2':'Ev_Name','EventGender':'Ev_Gender','Year':'Games_Year','Season':'Games_Season'})
ev.head()

,Ev_Sport,Ev_Name,Ev_Gender,Games_Year,Games_Season
0,Basketball,Basketball,Men's,1992,Summer
1,Judo,Extra-Lightweight,Men's,2012,Summer
2,Football,Football,Men's,1920,Summer
3,Tug-Of-War,Tug-Of-War,Men's,1900,Summer
4,Speed Skating,500 metres,Women's,1988,Winter


In [441]:
#need to capture the gameID from the game table: 
dfGames.head(1)


,GamesID,Games_City,Games_Season,Games_Year
0,1,Barcelona,Summer,1992


In [442]:
#merged_data= df1.merge(df2, on=["column1","column2"])

Eventdf = ev.merge(dfGames,  on=['Games_Year','Games_Season'])
Eventdf.sample(10)

,Ev_Sport,Ev_Name,Ev_Gender,Games_Year,Games_Season,GamesID,Games_City
119724,Cycling,Road Race Individual,Men's,2008,Summer,20,Beijing
110896,Ski Jumping,Large Hill Team,Men's,2006,Winter,19,Torino
191451,Shooting,Small-Bore Rifle Three Positions 50 metres,Men's,1988,Summer,29,Seoul
164611,Volleyball,Volleyball,Women's,1984,Summer,26,Los Angeles
11614,Basketball,Basketball,Women's,1992,Summer,1,Barcelona
145442,Shooting,Sporting Pistol 25 metres,Women's,2004,Summer,22,Athina
18240,Handball,Handball,Women's,2012,Summer,2,London
229524,Diving,Springboard,Men's,1928,Summer,35,Amsterdam
188221,Athletics,4 x 100 metres Relay,Women's,1972,Summer,28,Munich
94432,Gymnastics,Side Horse,Men's,1924,Summer,15,Paris


In [443]:
Eventdf=Eventdf[['Ev_Sport','Ev_Gender','Ev_Name','GamesID']]
Eventdf.head()

,Ev_Sport,Ev_Gender,Ev_Name,GamesID
0,Basketball,Men's,Basketball,1
1,Sailing,Men's,Two Person Dinghy,1
2,Equestrianism,Mixed,Jumping Individual,1
3,Wrestling,Men's,Heavyweight Freestyle,1
4,Judo,Men's,Half-Middleweight,1


In [444]:
Eventdf.drop_duplicates(inplace=True)
Eventdf.head()

,Ev_Sport,Ev_Gender,Ev_Name,GamesID
0,Basketball,Men's,Basketball,1
1,Sailing,Men's,Two Person Dinghy,1
2,Equestrianism,Mixed,Jumping Individual,1
3,Wrestling,Men's,Heavyweight Freestyle,1
4,Judo,Men's,Half-Middleweight,1


In [445]:
Eventdf['EvID'] = [i for i in range(1,len(Eventdf.index)+1)]
Eventdf=Eventdf[['EvID','Ev_Sport','Ev_Gender','Ev_Name','GamesID']] #reorder 
Eventdf.head()

,EvID,Ev_Sport,Ev_Gender,Ev_Name,GamesID
0,1,Basketball,Men's,Basketball,1
1,2,Sailing,Men's,Two Person Dinghy,1
2,3,Equestrianism,Mixed,Jumping Individual,1
3,4,Wrestling,Men's,Heavyweight Freestyle,1
4,5,Judo,Men's,Half-Middleweight,1


In [437]:
Eventdf.loc[Eventdf['GamesID']==52]

,EvID,Ev_Sport,Ev_Gender,Ev_Name,GamesID
209657,4609,Shooting,Men's,Trap,52
209659,4611,Cycling,Men's,Road Race Individual,52
209661,4613,Cycling,Men's,Road Race Team,52
209663,4615,Weightlifting,Men's,Lightweight,52
209665,4617,Swimming,Women's,4 x 100 metres Freestyle Relay,52
...,...,...,...,...,...
211393,4901,Wrestling,Men's,Heavyweight Freestyle,52
211935,4903,Wrestling,Men's,Featherweight Freestyle,52
211997,4905,Swimming,Women's,200 metres Breaststroke,52
212221,4907,Rowing,Men's,Single Sculls,52


In [446]:
Eventdf.tail()

,EvID,Ev_Sport,Ev_Gender,Ev_Name,GamesID
275960,6339,Athletics,Men's,Discus Throw,51
275962,6340,Fencing,Men's,Sabre Individual,51
276021,6341,Cycling,Men's,12-Hours Race,51
276028,6342,Swimming,Men's,500 metres Freestyle,51
276123,6343,Fencing,Men's,Foil Masters Individual,51


In [421]:
df.loc[df['City']=='Stockholm'].loc[df['Year']==1956]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender,Event2,Ath_FN,Ath_LN
6194,3479,Karl Vilhelm Nordstrm Ammitzbll,M,40,0,0,Denmark,DEN,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism Three-Day Event, Individual",None,Men's,Three-Day Event Individual,Karl,Ammitzbll
6195,3479,Karl Vilhelm Nordstrm Ammitzbll,M,40,0,0,Denmark,DEN,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism Three-Day Event, Team",None,Men's,Three-Day Event Team,Karl,Ammitzbll
6663,3728,Hans Christian Andersen,M,42,0,0,Denmark,DEN,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism Three-Day Event, Individual",None,Men's,Three-Day Event Individual,Hans,Andersen
6664,3728,Hans Christian Andersen,M,42,0,0,Denmark,DEN,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism Three-Day Event, Team",None,Men's,Three-Day Event Team,Hans,Andersen
8370,4608,Kaarlo Erkki Kustavi Anttinen (Andersin-),M,41,180,77,Finland,FIN,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism Three-Day Event, Individual",None,Men's,Three-Day Event Individual,Kaarlo,(Andersin-)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261838,130999,Hans Gnter Winkler,M,29,174,72,Germany,GER,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism ,Jumping, Team",Gold,Mixed,Jumping Team,Hans,Winkler
267427,133796,Mohamed Selim Zaki,M,31,0,0,Egypt,EGY,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism ,Jumping, Individual",None,Mixed,Jumping Individual,Mohamed,Zaki
267428,133796,Mohamed Selim Zaki,M,31,0,0,Egypt,EGY,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism ,Jumping, Team",None,Mixed,Jumping Team,Mohamed,Zaki
270202,135175,Hermann Zobel,M,47,0,0,Denmark,DEN,1956 Summer,1956,Summer,Stockholm,Equestrianism,"Equestrianism ,Dressage, Individual",None,Mixed,Dressage Individual,Hermann,Zobel


In [448]:
Eventdf.to_csv('/Users/isaiahhollars/Desktop/olympicProject/EventTable', sep='\t', encoding='utf-8', header=True, index=False)

# Final Boss: Make the Score Table 

In [475]:
df.head(1)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,EventGender,Event2,Ath_FN,Ath_LN
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,A,Dijiang


In [476]:
#merged_data= df1.merge(df2, on=["column1","column2"])
evgam = Eventdf.merge(dfGames, on=['GamesID'])
evgam.head(1)

,EvID,Ev_Sport,Ev_Gender,Ev_Name,GamesID,Games_City,Games_Season,Games_Year
0,1,Basketball,Men's,Basketball,1,Barcelona,Summer,1992


In [477]:
score = df.rename(columns={ 'Sport':'Ev_Sport','Event2':'Ev_Name','EventGender':'Ev_Gender','Year':'Games_Year','Season':'Games_Season'})
score.head(1)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Games_Year,Games_Season,City,Ev_Sport,Event,Medal,Ev_Gender,Ev_Name,Ath_FN,Ath_LN
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,A,Dijiang


In [478]:
score1 = score.merge(evgam, on=['Ev_Sport','Ev_Name','Ev_Gender','Games_Year','Games_Season'])

In [479]:
score.shape

(271116, 19)

In [480]:
score1.shape

(276243, 22)

In [481]:
score1.drop_duplicates(inplace=True)
score1.shape

(274858, 22)

In [482]:
score1.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Games_Year,Games_Season,City,Ev_Sport,Event,Medal,Ev_Gender,Ev_Name,Ath_FN,Ath_LN,EvID,GamesID,Games_City
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,A,Dijiang,1,1,Barcelona
1,2256,Vladan Alanovi,M,25,190,81,Croatia,CRO,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,Silver,Men's,Basketball,Vladan,Alanovi,1,1,Barcelona
2,2424,Santiago Aldama Aleson,M,23,213,98,Spain,ESP,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,Santiago,Aleson,1,1,Barcelona
3,4120,Armin Andres,M,33,180,76,Germany,GER,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,Armin,Andres,1,1,Barcelona
4,4140,Enrique Andreu Balbuena,M,24,210,102,Spain,ESP,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Basketball,None,Men's,Basketball,Enrique,Balbuena,1,1,Barcelona


In [483]:
score2= score1[['ID','EvID','Medal','NOC']]
score2=score2.rename(columns={'ID':'AthID','NOC':'TeamNOC'})
score2.head()


,AthID,EvID,Medal,TeamNOC
0,1,1,None,CHN
1,2256,1,Silver,CRO
2,2424,1,None,ESP
3,4120,1,None,GER
4,4140,1,None,ESP


In [484]:
score2.shape

(274858, 4)

In [485]:
score2.drop_duplicates(subset=['AthID','EvID'], inplace=True)
score2.shape

(274763, 4)

In [486]:
score2.head()

,AthID,EvID,Medal,TeamNOC
0,1,1,None,CHN
1,2256,1,Silver,CRO
2,2424,1,None,ESP
3,4120,1,None,GER
4,4140,1,None,ESP


In [487]:
score2.to_csv('/Users/isaiahhollars/Desktop/olympicProject/ScoreTable', sep='\t', encoding='utf-8', header=True, index=False)